In [ ]:
import cobra
import cobra.test
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import numpy.random as npr
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
import sys
if not '/Users/nlarusstone/Documents/MPhil/Research/cf_fba' in sys.path:
    sys.path.append('/Users/nlarusstone/Documents/MPhil/Research/cf_fba')
import src.utils as utils
%autoreload 2

In [ ]:
#model = cobra.io.load_json_model(filename='../models/ecoli_but.json')
model = cobra.io.read_sbml_model('../models/ecoli_cf_base.sbml')
#model = cobra.io.read_sbml_model('../models/iJO1366.xml')
mod_cf = model.copy()

In [ ]:
df = pd.read_csv('../data/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

In [ ]:
# TODO: Look into GoalEnv
class FBA_Rand_Env(gym.Env):      
    def __init__(self, model, df):
        self.model = model.copy()
        self.rxns = model.reactions
        self.n_rxns = len(self.rxns)
        # TODO Add metabs
        self.metabs = model.metabolites
        n_metabs = len(self.metabs)
        # LEARNING FROM EXPERIMENTAL DATA:
        self.df = df
        self.cond_scores = df['AVG.1']
        # Add 1 for not removing any
        self.action_space = spaces.MultiBinary(self.n_rxns) # TODO: Think about using MultiBinary
        #spaces.Dict({"reaction": spaces.Discrete(n_rxns + 1), "metabolite": spaces.Discrete(n_metabs + 1)})
        # Whether or not a reaction is present
        self.observation_space = spaces.MultiBinary(self.n_rxns)
        # In case they forget to reset
        self.state = np.ones(self.n_rxns, dtype=np.int8)
        self.time = 0
        
        self.unique_fluxes = {}
        
        self.log = []
        
        self._seed()
        #spaces.Dict({"reaction": spaces.Discrete(2), "metabolite": spaces.Discrete(3)})
        
    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _get_obs(self):
        return self.state
    
    def _take_action(self, state, action):
        # Returns new state
        return NotImplemented
    
    def _evaluate(self, state_rxns):
        return NotImplemented
    
    def reset(self):
        # TODO, choose random starting state
        self.state = np.ones(self.n_rxns, dtype=np.int8)
        self.last_action = None
        self.time = 0
        
        return self._get_obs()

    def step(self,action):
        # TODO: do better than this?
        new_state = self._take_action(self.state, action)
        self.state = new_state
        reward = self._evaluate(self.state)

        done = False
        #if self.time > 300:
        #    done = True

        self.time += 1
            
        return self.rxns[action[0]], reward, done, action

In [ ]:
# TODO: Look into GoalEnv
class FBA_Samp_Env(FBA_Rand_Env):  
    def _take_action(self, state, act):
        state = act
        return state
    
    def _evaluate(self, state_rxns):
        self.model.repair()
        # 1 represents a reaction to keep, so remove if not 1
        rxns_to_remove = [i for i,j in zip(self.model.reactions, state_rxns) if not j]
        with self.model as model:
            model.remove_reactions(reactions=rxns_to_remove, remove_orphans=False)
            objs = utils.add_addl_reactants(model, self.df)
            #objs, fluxes = utils.gen_fluxes_addl_reactants(self.cur_model, self.df)
            #obj, fluxes = utils.gen_fluxes(self.cur_model)
            #if not tuple(fluxes) in self.unique_fluxes:
            #  self.unique_fluxes[tuple(fluxes)] = state_rxns
            if not tuple(objs) in self.unique_fluxes:
                self.unique_fluxes[tuple(objs)] = state_rxns
            if sum(objs) < 0.01:
                return -1000
            corr = scipy.stats.spearmanr(objs, self.cond_scores)
        return corr

In [ ]:
class RandomAgent(object):
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done, prev_action):
        return self.action_space.sample()

In [ ]:
class AddBackAgent(object):
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done, prev_action):
        if reward < -1:
            return (prev_action[0], 'add')
        return (self.action_space.sample(), 'remove')

In [ ]:
env = FBA_Samp_Env(mod_cf, df)
agent = RandomAgent(env.action_space)
max_reward = (0, None)
prev_action, done = None, False
for i_episode in range(2):
    print 'Episode {0}'.format(i_episode)
    observation = env.reset()
    reward = 0
    for t in range(1001):
        #env.render()
        #print(observation)
        action = agent.act(observation, reward, done, prev_action)
        observation, reward, done, prev_action = env.step(action)
        if np.isnan(reward).any():
            print 'Nan', observation
            break
        if t % 10 == 0:
            print 'Time {0}, reward: {1}'.format(t, reward)
        if reward > max_reward[0]:
            max_reward = (reward, observation)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
print max_reward

In [ ]:
dis_rxns = []
i = 0
for objs, rxns in env.unique_fluxes.items():
    i += 1
    if sum(objs) < 0.01:
        continue
    dis_rxns.append(rxns)

In [ ]:
print i
print len(dis_rxns)

In [ ]:
max_len = (0, None)
for rxn in range(len(dis_rxns)):
    for flux_ser in dis_rxns[rxn][0]:
        if flux_ser.shape[0] > max_len[0]:
            max_len = (flux_ser.shape[0], flux_ser)
ind = max_len[1].index

In [ ]:
fluxes = []
for rxn in range(len(dis_rxns)):
    experiments = []
    for flux_ser in dis_rxns[rxn][0]:
        flux_ser_pad = flux_ser.reindex(ind)
        experiments.append(np.array(flux_ser_pad))
    experiment = np.stack(experiments, axis=0)
    fluxes.append(experiment)
flux_arr = np.stack(fluxes, axis=0)
flux_arr.shape

In [ ]:
flux_arr[0][0].shape

In [ ]:
np.save('../data/fluxes_ecoli_but', flux_arr)

In [ ]:
flux_arr_no_nan = np.nan_to_num(x=flux_arr)

In [ ]:
#def viz_flux_std(flux_arr):
stds = []
for exp in flux_arr_no_nan:
    for cond in exp:
        stds.append(np.std(cond))
#viz_flux_std(flux_arr)

In [ ]:
plt.hist(stds)
plt.show()

In [ ]:
# TODO: Look into GoalEnv
class FBA_Env(gym.Env):      
    def __init__(self, model):
        self.model = model
        self.rxns = model.reactions
        self.n_rxns = len(self.rxns)
        # TODO Add metabs
        self.metabs = model.metabolites
        n_metabs = len(self.metabs)
        # Add 1 for not removing any
        self.action_space = spaces.Discrete(n_rxns + 1) # TODO: Think about using MultiBinary
        #spaces.Dict({"reaction": spaces.Discrete(n_rxns + 1), "metabolite": spaces.Discrete(n_metabs + 1)})
        # Whether or not a reaction is present
        self.observation_space = spaces.MultiBinary(n_rxns + 1)
        self.state = self.observation_space.sample()
        self.time = 0
        
        self._seed()
        #spaces.Dict({"reaction": spaces.Discrete(2), "metabolite": spaces.Discrete(3)})

    def _evaluate(self, state_rxns):
        # 1 represents a reaction to keep, so remove if not 1
        rxns_to_remove = [i for i,j in zip(self.rxns, state_rxns) if not j]
        with self.model as model:
            model.remove_reactions(rxns_to_remove)
            obj = model.slim_optimize()
        return obj
        
    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    
    def step(self,action):
        cur_rxns = self.state        
        x = self.state
        reward = self._evaluate(cur_rxns)
        # TODO: do better than this?
        new_rxns = self.observation_space.sample()
        new_x = new_rxns

        self.state = new_x

        done = False
        if self.time > 300:
            done = True

        self.time += 1
            
        return self._get_obs(), reward, done, {}

    def reset(self):
        # self.step(np.array([0,0,0,0]))[0]
        # TODO, choose random starting state
        self.state = np.ones(self.n_rxns, dtype=np.int8)
        self.last_u = None
        self.time = 0
        
        return self._get_obs()

    def _get_obs(self):
        return self.state

    #def get_params(self):
    #    return self.A, self.B, self.Q, self.R